In [5]:
# Dependencies
import pandas as pd

In [6]:
# Import ZIP to ZCTA conversion table. All fields were imported as string to ensure the ZIP Codes retains the "00" prefixes 
zip_to_zcta_df = pd.read_excel("../00_input/zip_to_zcta_2019.xlsx", dtype = 'str')

In [7]:
# Check the ZIP/ZCTA dataframe data, particularly for retention of "00" for ZIP Codes
zip_to_zcta_df.head()

,ZIP_CODE,PO_NAME,STATE,ZIP_TYPE,ZCTA,zip_join_type
0,00501,Holtsville,NY,Post Office or large volume customer,11742,Spatial join to ZCTA
1,00544,Holtsville,NY,Post Office or large volume customer,11742,Spatial join to ZCTA
2,00601,Adjuntas,PR,Zip Code Area,00601,Zip Matches ZCTA
3,00602,Aguada,PR,Zip Code Area,00602,Zip Matches ZCTA
4,00603,Aguadilla,PR,Zip Code Area,00603,Zip Matches ZCTA


In [8]:
# Review data types in ZIP/ZCTA dataframe
zip_to_zcta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41107 entries, 0 to 41106
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ZIP_CODE       41107 non-null  object
 1   PO_NAME        41107 non-null  object
 2   STATE          41106 non-null  object
 3   ZIP_TYPE       41107 non-null  object
 4   ZCTA           41107 non-null  object
 5   zip_join_type  41107 non-null  object
dtypes: object(6)
memory usage: 1.9+ MB


In [9]:
# Check the minimum and maximum length of values in column ZIP_CODE
zip_code_len = zip_to_zcta_df["ZIP_CODE"].apply(len)
print(f"Max zip_code length is {zip_code_len.max()}.")
print(f"Min zip_code length is {zip_code_len.min()}.")

Max zip_code length is 5.
Min zip_code length is 5.


In [10]:
# Check the minimum and maximum length of values in column ZCTA
zcta_len = zip_to_zcta_df["ZCTA"].apply(len)
print(f"Max zcta length is {zcta_len.max()}.")
print(f"Min zcta length is {zcta_len.min()}.")

Max zcta length is 7.
Min zcta length is 5.


In [11]:
# A look into rows with length of values in column ZCTA > 5 
zip_to_zcta_df[zip_to_zcta_df.ZCTA.str.len() > zcta_len.min()]

,ZIP_CODE,PO_NAME,STATE,ZIP_TYPE,ZCTA,zip_join_type
41098,96939,Ngerulmud,PW,Zip Code Area,No ZCTA,"territory zip, no ZCTA available"
41099,96940,Koror,PW,Post Office or large volume customer,No ZCTA,"territory zip, no ZCTA available"
41100,96941,Pohnpei,FM,Post Office or large volume customer,No ZCTA,"territory zip, no ZCTA available"
41101,96942,Chuuk,FM,Post Office or large volume customer,No ZCTA,"territory zip, no ZCTA available"
41102,96943,Yap,FM,Post Office or large volume customer,No ZCTA,"territory zip, no ZCTA available"
41103,96944,Kosrae,FM,Post Office or large volume customer,No ZCTA,"territory zip, no ZCTA available"
41104,96960,Majuro,MH,Post Office or large volume customer,No ZCTA,"territory zip, no ZCTA available"
41105,96970,Ebeye,MH,Post Office or large volume customer,No ZCTA,"territory zip, no ZCTA available"
41106,96898,Minor Outlying Islands,NaN,Post Office or large volume customer,No ZCTA,"territory zip, no ZCTA available"


In [12]:
# Remove all rows with ZCTA = "No ZCTA"
zip_to_zcta_df = zip_to_zcta_df[~(zip_to_zcta_df.ZCTA.str.len() > zcta_len.min())]

In [13]:
# Overview of the dataset after the removal
zip_to_zcta_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41098 entries, 0 to 41097
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ZIP_CODE       41098 non-null  object
 1   PO_NAME        41098 non-null  object
 2   STATE          41098 non-null  object
 3   ZIP_TYPE       41098 non-null  object
 4   ZCTA           41098 non-null  object
 5   zip_join_type  41098 non-null  object
dtypes: object(6)
memory usage: 2.2+ MB


In [14]:
# Count the number of unique zip_codes

zip_to_zcta_df['ZIP_CODE'].nunique()

41098

In [15]:
# Count the number of unique ZCTAs

zip_to_zcta_df['ZCTA'].nunique()

33166

As there are more zip_codes than ZCTA, we suspect there are some ZCTAs that are matched against more than one zip_codes.

In [16]:
# Overview of the duplicated ZCTAs
duplicate_zcta_rows = zip_to_zcta_df[zip_to_zcta_df.duplicated(subset=["ZCTA"], keep=False)].sort_values(by="ZCTA")
duplicate_zcta_rows

,ZIP_CODE,PO_NAME,STATE,ZIP_TYPE,ZCTA,zip_join_type
4,00603,Aguadilla,PR,Zip Code Area,00603,Zip Matches ZCTA
5,00604,Aguadilla,PR,Post Office or large volume customer,00603,Spatial join to ZCTA
6,00605,Aguadilla,PR,Post Office or large volume customer,00603,Spatial join to ZCTA
10,00612,Arecibo,PR,Zip Code Area,00612,Zip Matches ZCTA
11,00613,Arecibo,PR,Post Office or large volume customer,00612,Spatial join to ZCTA
...,...,...,...,...,...,...
41066,99850,Juneau,AK,Post Office or large volume customer,99801,Spatial join to ZCTA
41077,99928,Ward Cove,AK,Post Office or large volume customer,99901,Spatial join to ZCTA
41067,99901,Ketchikan,AK,Zip Code Area,99901,Zip Matches ZCTA
41071,99921,Craig,AK,Zip Code Area,99921,Zip Matches ZCTA


> To ensure the flexibility in the zip_code and ZCTA mapping, we'll keep the relationship between zip in the zip_code data and ZCTA in the census data as many_to_many. This junction table of zip and ZCTA will have two composite primary keys to map ZCTA in the census data with zip_code in the zip_code data.

In [17]:
# Export a zip_to_zcta table with zip_code, po_name, state, ZCTA to match against zip_code, city and state in the zip_code table
short_zip_zcta = zip_to_zcta_df[["ZIP_CODE", "PO_NAME", "STATE", "ZCTA"]].rename(columns={"ZIP_CODE": "zip_code"})
short_zip_zcta.head()

,zip_code,PO_NAME,STATE,ZCTA
0,00501,Holtsville,NY,11742
1,00544,Holtsville,NY,11742
2,00601,Adjuntas,PR,00601
3,00602,Aguada,PR,00602
4,00603,Aguadilla,PR,00603


In [18]:
short_zip_zcta.to_csv("zcta_to_match.csv", index=False)

In [19]:
# Export a final zip_to_zcta table to load into the database
final_zcta = short_zip_zcta[["zip_code", "ZCTA"]]
final_zcta.head()

,zip_code,ZCTA
0,00501,11742
1,00544,11742
2,00601,00601
3,00602,00602
4,00603,00603


In [20]:
final_zcta.to_csv("zcta.csv", index=False)